In [61]:
# TODO
# Overleaf Abgabebereit machen

# Auf dem Schirm behalten
# Augmentierung/Generalisierung
# Encoder Einbinden (Efficient Net einbinden)
# Hyper-Parameter Tuning

print('   ')
print('                                                  ####################################################################')
print('                                                  #######-------->>>> ***++++  START  +++++*** <<<<-----------########')
print('                                                  ####################################################################')
print('   ')


import time
import os
import numpy as np
import pdb
import librosa
from scipy.signal import stft
import librosa.display
#from metrics import mse
import tensorflow as tf
from keras.optimizers import Adam
from make_network.system import get_model
from make_network.losses import MEL_MR_L1_Loss
from make_network.load_training_data import get_train_val_Data, get_input_shape
from make_network.callbacks import LRTensorBoard, get_run_logdir, SaveMelSpectrogramCallback
from helpers.utils import get_file_paths
#from make_network.callbacks import PrintControllerOutputCallback
import keras_tuner as kt


# Load file paths 
x_dir = './datasets/test/dirty'
y_dir = './datasets/test/clean'

AUTOTUNE = tf.data.AUTOTUNE  # Use tf.data.AUTOTUNE

   
                                                  ####################################################################
                                                  #######-------->>>> ***++++  START  +++++*** <<<<-----------########
                                                  ####################################################################
   


In [62]:
config = {
    'frame_length': 512,     # Frame step for STFT
    'batch_size': 64,       # z.B.: 30, da wir 150 Tracks haben
    'random_seed': 42,
    'train_ratio': 0.8,
    'test_ratio' : 0.1,
    'val_ratio' : 0.1,
    'buffer_size' : 1500,   ## for now
    'std_sample_rate' : 24000,
    'fft_length' : 512,  # Increase for better frequency resolution
    'hop_length' : 256, # Common choice for 16kHz sample rate, adjust as needed
    'n_mels' : 256, # Decrease if you get empty filter warnings, or to reduce computation
    'lr' : 0.01,
    'lr_decay': 0.97,
    'limit_tracks' : True,
    'max_tracks' : 250,
    'no_epochs' : 10,
    'system' : "Apple_Silicon", # Intel/AMD
    'l1_loss_ratio' : 1,
    'num_control_params': 24
}

In [63]:
# Load file paths 
x_dir = './datasets/test/dirty'
y_dir = './datasets/test/clean'

print(x_dir)
# Get Train Data

train_dataset, val_dataset, steps_per_epoch, validation_steps = get_train_val_Data(x_dir, y_dir, config)

total_size = len(get_file_paths(x_dir)*2)
train_size = int(total_size * config['train_ratio'])
test_size = int(total_size * config['test_ratio'])
val_size = total_size - train_size - test_size

input_shape = get_input_shape(train_dataset)


./datasets/test/dirty
<--- x_files --->
['./datasets/test/dirty/585838_neutral_IR.wav', './datasets/test/dirty/582670_bright_IR.wav', './datasets/test/dirty/1198542_broadcast_IR.wav', './datasets/test/dirty/1420417_bright_IR.wav', './datasets/test/dirty/1042762_neutral_IR.wav', './datasets/test/dirty/1003417_neutral_IR.wav', './datasets/test/dirty/1135470_bright_IR.wav', './datasets/test/dirty/1052321_neutral_IR.wav', './datasets/test/dirty/1136338_broadcast_IR.wav', './datasets/test/dirty/976777_bright_IR.wav']
Steps per epoch: 1, Validation steps: 1
Total size: 72, Train size: 57, Validation size: 8
x_train shape: (57, 60000)
y_train shape: (57, 60000)
y_train_targets shape: (57, 60000)
<-> printing input shape here <->
(57, 60000)


In [64]:
def hyperparameter_model(hp):
    

    # Set the hp_search parameters
    hp_lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log') 
    hp_opt = hp.Choice('optimizer', values = ['sgd', 'momentum', 'adam'])
    hp_lr_schedule = hp.Boolean('lr_schedule', default=False)
    hp_l1_loss_ratio = hp.Float('l1_ratio', min_value=0.1, max_value=10, sampling='linear')
    hp_hidden = hp.Choice('hidden', values=[32, 128, 256, 512])
    hp_fft_length = hp.Choice('fft_lengths', values=[256, 512, 1024])
    

    # check configs
    config = {
    'frame_length': 512,     # Frame step for STFT
    'batch_size': 64,       # z.B.: 30, da wir 150 Tracks haben
    'random_seed': 42,
    'train_ratio': 0.8,
    'test_ratio' : 0.1,
    'val_ratio' : 0.1,
    'buffer_size' : 1500,   ## for now
    'std_sample_rate' : 24000,
    'fft_length' : hp_fft_length,  # Increase for better frequency resolution
    'hop_length' : 256, # Common choice for 16kHz sample rate, adjust as needed
    'n_mels' : 256, # Decrease if you get empty filter warnings, or to reduce computation
    'lr' : 0.01,
    'lr_decay': 0.97,
    'limit_tracks' : True,
    'max_tracks' : 250,
    'no_epochs' : 15,
    'system' : "Apple_Silicon", # Intel/AMD
    'l1_loss_ratio' : 1,
    'hidden_dim': hp_hidden, # default value in StyleController
    'num_control_params': 24
    }



    # Get model architecture 
    model, _ = get_model(config, input_shape)  


    # check l1-loss-ratio
    mel_mr_l1 = MEL_MR_L1_Loss(l1_ratio=hp_l1_loss_ratio) 
    
    # check if schedule active
    if hp_lr_schedule:
        lr = tf.keras.optimizers.schedules.ExponentialDecay(hp_lr, decay_steps=150, decay_rate=0.97, staircase=True)
    else:
        lr = hp_lr
    
    # check which optimizer to get
    if hp_opt == 'sgd':
        opt = tf.keras.optimizers.legacy.SGD(learning_rate=lr, momentum=0.0)
    elif hp_opt == 'momentum':
        opt = tf.keras.optimizers.legacy.SGD(learning_rate=lr, momentum=0.9)
    elif hp_opt == 'adam':
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)

    # compile
    model.compile(optimizer=opt,loss=mel_mr_l1) 

    return model

In [65]:
 # empty gridsearch and log folder
! rm -rf _gridsearch/*
! rm -rf _logs/*

zsh:1: no matches found: _logs/*


In [66]:
tuner = kt.RandomSearch(
            hyperparameter_model,
            objective=kt.Objective('val_loss', direction='min'),
            seed=42,
            max_trials=25,
            executions_per_trial=10,
            directory = '_gridsearch',)

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_data_channel (InputL  [(None, 60000)]              0         []                            
 ayer)                                                                                            
                                                                                                  
 referenz_data_channel (Inp  [(None, 60000)]              0         []                            
 utLayer)                                                                                         
                                                                                                  
 MelLayer_in (MelSpectrogra  (None, 256, 256, 1)          0         ['input_data_channel[0][0]']  
 mLayer)                                                                                    

In [67]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'momentum', 'adam'], 'ordered': False}
lr_schedule (Boolean)
{'default': False, 'conditions': []}
l1_ratio (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 10.0, 'step': None, 'sampling': 'linear'}
hidden (Choice)
{'default': 32, 'conditions': [], 'values': [32, 128, 256, 512], 'ordered': True}
fft_lengths (Choice)
{'default': 256, 'conditions': [], 'values': [256, 512, 1024], 'ordered': True}


In [68]:
run_logdir = get_run_logdir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch='5,10')
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(run_logdir, "cp-{epoch:04d}.weights.h5"), save_weights_only=True, verbose=1)

lr_filepath = os.path.join(run_logdir, "lr-{epoch:04d}.json")
lr_saving_cb = LRTensorBoard(log_dir=run_logdir)

2024-03-22 20:29:29.836974: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-22 20:29:29.836995: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-03-22 20:29:29.837761: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [69]:
tuner.search(train_dataset, 
             epochs=config['no_epochs'],
             steps_per_epoch=steps_per_epoch,
             validation_data=val_dataset,
             validation_steps=validation_steps,
             callbacks=[tensorboard_cb, cp_callback, lr_saving_cb])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.0019004         |0.0019004         |learning_rate
sgd               |sgd               |optimizer
False             |False             |lr_schedule
2.7916            |2.7916            |l1_ratio
512               |512               |hidden
512               |512               |fft_lengths

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_data_channel (InputL  [(None, 60000)]              0         []                            
 ayer)                                                                                            
                                                                                                  
 referenz_data_channel (Inp  [(None, 60000)]              0         []                            
 utLayer)       

2024-03-22 20:32:30.284058: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-22 20:32:30.284066: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


1/1 [==============================] - ETA: 0s - loss: 1.8018
Epoch 5: saving model to my_logs/run_2024_03_22_20_29_29/cp-0005.weights.h5
1/1 [==============================] - 42s 42s/step - loss: 1.8018 - val_loss: 141.3078 - lr: 0.0019
Epoch 6/10
1/1 [==============================] - ETA: 0s - loss: 1.7330
Epoch 6: saving model to my_logs/run_2024_03_22_20_29_29/cp-0006.weights.h5
1/1 [==============================] - 44s 44s/step - loss: 1.7330 - val_loss: 53.3991 - lr: 0.0019
Epoch 7/10
1/1 [==============================] - ETA: 0s - loss: 1.6930
Epoch 7: saving model to my_logs/run_2024_03_22_20_29_29/cp-0007.weights.h5
1/1 [==============================] - 42s 42s/step - loss: 1.6930 - val_loss: 32.6880 - lr: 0.0019
Epoch 8/10
1/1 [==============================] - ETA: 0s - loss: 1.6537
Epoch 8: saving model to my_logs/run_2024_03_22_20_29_29/cp-0008.weights.h5
1/1 [==============================] - 42s 42s/step - loss: 1.6537 - val_loss: 20.6014 - lr: 0.0019
Epoch 9/10
1/1

2024-03-22 20:36:47.209344: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-03-22 20:36:56.339972: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-03-22 20:36:56.344073: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: my_logs/run_2024_03_22_20_29_29/00/execution0/plugins/profile/2024_03_22_20_36_56/Constantins-MacBook-Pro.local.xplane.pb


1/1 [==============================] - ETA: 0s - loss: 1.6085
Epoch 10: saving model to my_logs/run_2024_03_22_20_29_29/cp-0010.weights.h5
1/1 [==============================] - 54s 54s/step - loss: 1.6085 - val_loss: 11.4112 - lr: 0.0019
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_data_channel (InputL  [(None, 60000)]              0         []                            
 ayer)                                                                                            
                                                                                                  
 referenz_data_channel (Inp  [(None, 60000)]              0         []                            
 utLayer)                                                                                         
                                                   

...

2024-03-22 21:13:35.650556: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-03-22 21:13:44.566968: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-03-22 21:13:44.570403: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: my_logs/run_2024_03_22_20_29_29/00/execution5/plugins/profile/2024_03_22_21_13_44/Constantins-MBP.xplane.pb


1/1 [==============================] - ETA: 0s - loss: 1.5668
Epoch 10: saving model to my_logs/run_2024_03_22_20_29_29/cp-0010.weights.h5
1/1 [==============================] - 53s 53s/step - loss: 1.5668 - val_loss: 4.9519 - lr: 0.0019
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_data_channel (InputL  [(None, 60000)]              0         []                            
 ayer)                                                                                            
                                                                                                  
 referenz_data_channel (Inp  [(None, 60000)]              0         []                            
 utLayer)                                                                                         
                                                    

2024-03-22 21:16:54.639782: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-22 21:16:54.639797: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


1/1 [==============================] - ETA: 0s - loss: 1.9905
Epoch 5: saving model to my_logs/run_2024_03_22_20_29_29/cp-0005.weights.h5
1/1 [==============================] - 44s 44s/step - loss: 1.9905 - val_loss: 1.9273 - lr: 0.0019
Epoch 6/10
1/1 [==============================] - ETA: 0s - loss: 1.9408
Epoch 6: saving model to my_logs/run_2024_03_22_20_29_29/cp-0006.weights.h5
1/1 [==============================] - 44s 44s/step - loss: 1.9408 - val_loss: 1.9186 - lr: 0.0019
Epoch 7/10
1/1 [==============================] - ETA: 0s - loss: 1.8766
Epoch 7: saving model to my_logs/run_2024_03_22_20_29_29/cp-0007.weights.h5
1/1 [==============================] - 44s 44s/step - loss: 1.8766 - val_loss: 1.8868 - lr: 0.0019
Epoch 8/10
1/1 [==============================] - ETA: 0s - loss: 1.8314
Epoch 8: saving model to my_logs/run_2024_03_22_20_29_29/cp-0008.weights.h5
1/1 [==============================] - 42s 42s/step - loss: 1.8314 - val_loss: 1.8890 - lr: 0.0019
Epoch 9/10
1/1 [===

2024-03-22 21:21:16.434301: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)